## Comments about Assignment 2
#### Adriano Mundo 10524163 , Mario Sacaj 10521887

Assignment 2 for the ANN2DL course was to participate a competition on kaggle with the objective to develop a Neural Network Architecture for Image Segmentation.

We initially tried to adapt the CNN Segmentation model presented in the Lab: we set the random SEED, used the "validation_split" attribute in order to automatically generate the validation data, we applied some data augumentation, we set the loss to "binary crossentropy" as we thought it suited more the classification problem we had and adapted the model structure so as to output only one value for pixel in the [0,1] range. 
It however performed poorly. So, after searching a bit through the Internet we found a Segmentation classificator called **MultiResUNet**, in which **UNet**'s skip connections were replaced with *Res* paths.

Paper: https://arxiv.org/pdf/1902.04049.pdf

Proposed implementation: https://github.com/nibtehaz/MultiResUNet

We adapted the proposed implementation to our domain and we in fact experienced a jump in *val_My_IoU* score (it went from 20 something % to 55 %, achieved in just one epoch). It however stuck at that value and then started to decay as the epochs were passing. Fine tuning hyperparameters like the learnig rate or the optimizer didn't help.
So we changed again our structure to a classic **UNet** model, adapting the implementation proposed here:

https://github.com/zhixuhao/unet

It converged more slowly than MultiResUNet but outperformed it. Finetuning the data augumentation parameters and letting it train for 40 epochs allowed us to achieve a even better result.

In [ ]:
import os
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
import json

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Set Dataset dir
dataset_dir = '/kaggle/input/ann-and-dl-image-segmentation/Segmentation_Dataset'

# If you want to load your model from disk set True
restore_model = False

# If you want to commit set to False
not_commit = True

# Get current working directory
cwd = os.getcwd()

# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
# SPECIFIC PARAMETERS

# Batch size
bs = 4

# img shape
img_h = 256
img_w = 256

# clsses 0,1 building and background
num_classes=2

In [ ]:
# IMAGE GENERATOR CREATION WITH DATA AUGUMENTATION

apply_data_augmentation = True

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    
    train_img_data_gen = ImageDataGenerator(rotation_range=10,
                                            width_shift_range=10,
                                            height_shift_range=10,
                                            zoom_range=0.3,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            fill_mode='constant',
                                            cval=0,
                                            rescale=1./255,
                                           validation_split = 0.2)
    
    train_mask_data_gen = ImageDataGenerator(rotation_range=10,
                                             width_shift_range=10,
                                             height_shift_range=10,
                                             zoom_range=0.3,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             fill_mode='constant',
                                             cval=0,
                                            rescale=1./255,
                                            validation_split = 0.2)
else:
    
    train_img_data_gen = ImageDataGenerator(rescale=1./255)
    train_mask_data_gen = ImageDataGenerator(rescale=1./255)

# Create test ImageDataGenerator object
test_img_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# IMG DATA GENERATOR FLOW FROM DIRECTORY

# Training
# Two different generators for images and masks
# Same SEED to train and validation generator
training_dir = os.path.join(dataset_dir, 'training')

train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs,
                                                       color_mode='rgb',
                                                       class_mode=None,
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       subset='training')  

train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         color_mode='grayscale',
                                                         class_mode=None,
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         seed=SEED,
                                                        subset='training')

train_gen = zip(train_img_gen, train_mask_gen)

# Validation 
valid_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs,
                                                       color_mode='rgb',
                                                       class_mode=None, 
                                                       shuffle=False,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       subset='validation')

valid_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         color_mode='grayscale',
                                                         class_mode=None, 
                                                         shuffle=False,
                                                         interpolation='bilinear',
                                                         seed=SEED,
                                                         subset='validation')

valid_gen = zip(valid_img_gen, valid_mask_gen)

# Test
test_dir = os.path.join(dataset_dir, 'test')

test_img_gen = test_img_data_gen.flow_from_directory(os.path.join(test_dir, 'images'),
                                                     target_size=(img_h, img_w),
                                                     batch_size=bs,
                                                     color_mode='rgb',
                                                     class_mode=None, 
                                                     shuffle=False,
                                                     interpolation='bilinear',
                                                     seed=SEED)

test_gen = test_img_gen

In [ ]:
# UTILS FUNCTIONS

# Casting float values to int 
def prepare_target(x_, y_):
    y_ = tf.cast(y_, tf.int32)
    return x_, y_

# Evaluation Metric
def my_IoU(y_true, y_pred):
    
    # from pobability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32) # when using sigmoid. Use argmax for softmax

    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    
    # IoU
    return intersection / union

# Save the model's weights
def saveModel(model):

    model_json = model.to_json()
    
    fp = open('modelP.json','w')
    fp.write(model_json)
    
    model.save_weights('modelW.h5')

In [ ]:
# DATASET OBJECT CREATION AND REPEAT

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))

train_dataset = train_dataset.map(prepare_target)

train_dataset = train_dataset.repeat()

# Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))

valid_dataset = valid_dataset.map(prepare_target)

valid_dataset = valid_dataset.repeat()

In [ ]:
# HYPERPARAMETERS

# Loss
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False) 

# Learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate = lr)

# Validation metrics
metrics = [my_IoU]

In [ ]:
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


def unet(metrics, input_size = (256,256,3)):

    ##ENCODER
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)


    ## DECODER
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = metrics)
    return model


In [ ]:
# MODEL INSTANCE

modelUnet = unet(metrics)

# MODEL COMPILING -> UNET is already compiled
#model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# CALLBACKS
cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'segmentation_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

In [ ]:
# EARLY STOPPING
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    callbacks.append(es_callback)

In [ ]:
# MODEL TRAINING
        
modelUnet.fit_generator(train_gen,
            epochs=40,  #### set repeat in training dataset
            steps_per_epoch=len(train_img_gen),
            validation_data=valid_gen,
            validation_steps=len(valid_img_gen), 
            callbacks=callbacks)

saveModel(modelUnet)
    

In [ ]:
# ENCODING OF OUTPUT FOR SUBMISSION
def rle_encode(img):
      # Flatten column-wise
      pixels = img.T.flatten()
      pixels = np.concatenate([[0], pixels, [0]])
      runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
      runs[1::2] -= runs[::2]
      return ' '.join(str(x) for x in runs)

In [ ]:
# RESULTS FILE CREATION
def create_csv(results, results_dir=''):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:

      f.write('ImageId,EncodedPixels,Width,Height\n')

      for key, value in results.items():
          f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')

In [ ]:
# COMPETITION SCORE 
def competition_score(y_true, y_pred):
    score = my_IoU(y_true, y_pred)
    thresholds = np.arange(0.5, 1.0, 0.05)
    competition_score = 0

    for t in thresholds:
        if score > t:
            competition_score += 1

    competition_score /= len(thresholds)

    return competition_score

In [ ]:
def prepare_test(x_):
    y_ = tf.cast(x_ > 0.5, tf.int32)
    return y_

In [ ]:
if not_commit:  
    preds = modelUnet.predict_generator(test_gen)
    preds = prepare_test(preds)
    preds = np.array(preds)

In [ ]:
# CREATE CSV 
image_filenames = test_gen.filenames
image_filenames
results = {}
if not_commit:
    for i in range(len(image_filenames)):
       results[(image_filenames[i][4:-4])] = rle_encode(preds[i])

    create_csv(results)